# Model 1: MultiNomial Naive Bayes

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import re


## Loading the Dataset

In [5]:
data = pd.read_csv('/Users/tahafaisal/Desktop/ml-news-classification/data/FINAL_DATASET.csv')

data.info()
print()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2185 entries, 0 to 2184
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2185 non-null   int64 
 1   title       2185 non-null   object
 2   link        2185 non-null   object
 3   content     2185 non-null   object
 4   gold_label  2185 non-null   object
dtypes: int64(1), object(4)
memory usage: 85.5+ KB



,id,title,link,content,gold_label
0,0,ادارہ 'علم دوست' کی جانب سے'معلوماتِ عامہ' کے ...,https://www.express.pk/story/2733338/idara-ilm...,ادارہ 'علم دوست' کی جانب سے'معلوماتِ عامہ' کے ...,Entertainment
1,1,فلم ساز ودھو ونود چوپڑا کی نئی فلم ’زیرو سے ری...,https://www.express.pk/story/2733336/director-...,معروف فلم ساز ودھو ونود چوپڑا نے اپنی نئی فلم ...,Entertainment
2,2,عمر بڑھنے کے ساتھ وزن کم کرنا مشکل ہوتا ہے، اب...,https://www.express.pk/story/2733331/umer-barh...,ابھیشیک بچن نے اپنی نئی فلم ’آئی وانٹ ٹو ٹاک‘ ...,Entertainment
3,3,ملائکہ اروڑا والد کے انتقال کے بعد کام پر واپس...,https://www.express.pk/story/2733327/malaikaar...,مشہور اداکارہ ملائکہ اروڑا حال ہی میں والد کے ...,Entertainment
4,4,ڈائریکٹر دھرمیش درشن کا دو بار شاہ رخ خان کی ف...,https://www.express.pk/story/2733325/directord...,بالی ووڈ کے معروف ہدایتکار دھرمیش درشن نے حالی...,Entertainment


## Preprocessing the Dataset

In [6]:
with open('../data/stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    return text

def normalize_unicode(text):
    return re.sub(r'[٠١٢٣٤٥٦٧٨٩]', '', text)  # Remove Urdu numerals

def normalize_urdu(text):
    text = re.sub(r'[؁؂؃؄؅؆؇؈؉؊؋،؛؟]', '', text)  # Remove Urdu punctuation
    text = re.sub(r'[آإأٱ]', 'ا', text)  # Normalize different forms of 'alif'
    text = re.sub(r'[ىېۍ]', 'ی', text)  # Normalize different forms of 'ye'
    text = re.sub(r'[ۀہ]', 'ہ', text)  # Normalize 'heh'
    text = re.sub(r'[ؤو]', 'و', text)  # Normalize 'waw'
    text = re.sub(r'[ءئ]', 'ی', text)  # Normalize 'hamza' with 'ye'
    return text

def tokenize_text(text):
    return re.findall(r'\w+', text)  # Extract words using regex

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

def lemmatize_custom(word):
    if word.endswith('نا') or word.endswith('تے'):
        return word[:-2]  # Strip suffix
    elif word.endswith('ا') or word.endswith('ی'):
        return word[:-1]  # Remove singular/plural suffix
    return word

def lemmatize_text(tokens):
    return [lemmatize_custom(word) for word in tokens]

def remove_non_informative_words(tokens):
    return [word for word in tokens if len(word) > 1]  # Remove single-character tokens


def preprocess_urdu_text(text):
    text = clean_text(text)  # Clean text
    text = normalize_unicode(text)  # Normalize Unicode
    text = normalize_urdu(text)  # Normalize Urdu-specific characters
    tokens = tokenize_text(text)  # Tokenize text
    tokens = remove_stopwords(tokens)  # Remove stopwords
    tokens = lemmatize_text(tokens)
    tokens = remove_non_informative_words(tokens)# Apply lemmatization
    return ' '.join(tokens)  # Return preprocessed text

# Apply preprocessing to dataset
data['title'] = data['title'].apply(preprocess_urdu_text)
data['content'] = data['content'].apply(preprocess_urdu_text)
data['combined'] = data['title'] + " " + data['content']


In [ ]:
X = data['content'] 
y = data['gold_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1748,) (1748,)
(437,) (437,)


## Implementation of Multinomial Naive Bayes From Scratch

In [8]:
class BagOfWords:
    def __init__(self):
        self.vocabulary = {}  
        self.vocab_size = 0

    def fit(self, corpus):

        unique_words = set()

        for sentence in corpus:
            words = sentence.split() 
            unique_words.update(words)  

        self.vocabulary = {word: idx for idx, word in enumerate(sorted(unique_words))}
        self.vocab_size = len(self.vocabulary)

    def vectorize(self, sentence):

        vector = [0] * self.vocab_size
        
        words = sentence.split()

        for word in words:
            if word in self.vocabulary:
                index = self.vocabulary[word]
                vector[index] += 1

        return vector

In [9]:
model = BagOfWords()
model.fit(X_train)

vector_training = [model.vectorize(doc) for doc in X_train]
vector_test = [model.vectorize(doc) for doc in X_test]


In [10]:
class MultiNaiveBayes:
    def __init__(self):
        self.classprobs = {}  
        self.wordprobs = {}  
        self.vocab_size = 0    
        self.bow = BagOfWords()  # Assume BagOfWords handles vectorization without LabelEncoder                

    def fit(self, X, y):
        self.bow.fit(X)         
        X_vectorized = [self.bow.vectorize(doc) for doc in X] 
        self.vocab_size = self.bow.vocab_size 
        n_docs = len(X)
        
        # Get unique classes as they are
        unique_classes = np.unique(y)  
        class_counts = {c: 0 for c in unique_classes}
        word_counts = {c: np.zeros(self.vocab_size) for c in unique_classes}

        for i in range(n_docs):
            c = y.iloc[i]  # Updated to y.iloc[i] to handle pandas Series correctly
            class_counts[c] += 1
            word_counts[c] += X_vectorized[i]

        for c in unique_classes:
            self.classprobs[c] = class_counts[c] / n_docs
            total_words_in_class = np.sum(word_counts[c])
            self.wordprobs[c] = (word_counts[c] + 1) / (total_words_in_class + self.vocab_size)
    
    def predict(self, X):
        X_vectorized = np.array([self.bow.vectorize(doc) for doc in X])
        predictions = []
        
        for doc_vec in X_vectorized:
            class_scores = {}
            for c in self.classprobs:
                log_prob_c = np.log(self.classprobs[c])
                log_prob_x_given_c = np.sum(doc_vec * np.log(self.wordprobs[c]) + (1 - doc_vec) * np.log(1 - self.wordprobs[c]))
                class_scores[c] = log_prob_c + log_prob_x_given_c
            
            best_class = max(class_scores, key=class_scores.get)
            predictions.append(best_class)
        
        return predictions


In [11]:
model = MultiNaiveBayes()
model.fit(X_train, y_train) 

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
confusionmatrix = confusion_matrix(y_test, predictions)

print("Manually Implemented Naive Bayes:")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print(confusionmatrix)

Manually Implemented Naive Bayes:
Accuracy:  0.9862700228832952
Precision:  0.9857011494252873
Recall:  0.9850844561770152
F1 Score:  0.9853792307812949
[[ 74   0   0   2   0]
 [  0  86   1   0   0]
 [  0   1  98   0   0]
 [  1   0   1  73   0]
 [  0   0   0   0 100]]


In [12]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(vector_training, y_train)
mnbprediction = model.predict(vector_test)

accuracy = accuracy_score(y_test, mnbprediction)
precision = precision_score(y_test, mnbprediction, average='macro')
recall = recall_score(y_test, mnbprediction, average='macro')
f1 = f1_score(y_test, mnbprediction, average='macro')
confusionmatrix = confusion_matrix(y_test, mnbprediction)

print("For Sklearn MultiNomial Naive Bayes:")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print(confusionmatrix)
print()

For Sklearn MultiNomial Naive Bayes:
Accuracy:  0.9862700228832952
Precision:  0.9857011494252873
Recall:  0.9850844561770152
F1 Score:  0.9853792307812949
[[ 74   0   0   2   0]
 [  0  86   1   0   0]
 [  0   1  98   0   0]
 [  1   0   1  73   0]
 [  0   0   0   0 100]]

